In [3]:
#####
#preliminary functions
#####

using Printf
using Plots
using LinearAlgebra
using Statistics

######
#WA mc routine

const L       = 10             # linear size of lattice
const temps   = 1.2:-0.2:0.1  # temperatures to sample
const nt = length(temps)      #number of temperatures
#the 4 different bond directions on the lattice
#index 1,2 is the direction in x,y 
#index 3 is the default link value in that direction
#link_default = [1,1,-1,-1]



#these arrays allow us to selct the x or y direction bonds
all_x = ones(Int32, 2*L, 2*L)
all_y = ones(Int32, 2*L, 2*L)

for i in 1:2:2*L
    all_x[:,i] = zeros(Int32, 2*L)
    all_y[i,:] = zeros(Int32, 2*L)
end

In [17]:
#######
#third attempts after Kun's insight
#######

dir_list = [[[1,0],1], [[0,1],1], [[-1,0],-1], [[0,-1,],-1]] 
#index 1,2 is the direction in x,y 
#index 3 is the default link value in that direction
#link_default = [1,1,-1,-1]



#these arrays allow us to selct the x or y direction bonds
all_x = ones(Int32, 2*L, 2*L)
all_y = ones(Int32, 2*L, 2*L)

for i in 1:2:2*L
    all_x[:,i] = zeros(Int32, 2*L)
    all_y[i,:] = zeros(Int32, 2*L)
end

function mod_latt_plus(site, direction_shift)
    val = ones(Int32, 2) + mod.(site - ones(Int32, 2) + direction_shift, 2*L)
    return val
end

#this function takes in a site M, the map of bonds and the "ratio"
#and return a new M based on the metropolis acceptance ratio
function move_worm(Ms, all_map, ratio)
    dir_link, prop_add = dir_list[rand(1:4)] #random link direction and change in bond value
                
    link_v = mod_latt_plus(Ms, dir_link)
    link_val = all_map[CartesianIndex(link_v[1], link_v[2])] #that's the current bond value
    new_val = link_val + prop_add #new bond value
    deltaS = (ratio)*(new_val^2 - link_val^2) 


    #set the barrier height
    barrier = minimum([1, exp(-deltaS)])
    prob = rand()
    if (prob < barrier)
        #if you accept the move
        all_map[CartesianIndex(link_v[1], link_v[2])] = new_val
        Ms = mod_latt_plus(Ms, 2*dir_link)
        
        #the following lines were moved to the global function
        #dist_M_to_I += dir_link
        #gs_map = ones(Int32, 2) + mod.(dist_M_to_I, L)
        #g_map[CartesianIndex(gs_map[1], gs_map[2])] += 1
    else
        dir_link = [0,0]
    end
    
    return Ms, dir_link, all_map
end



function sweep_v3(nv, ratio, all_map, kappa) 
    
    ci = 0
    
    w_list = []
    accum_w_list = [] #this keeps the accumulated winding number in memory
    accum_winding = 0 #this accumulates the winding number
    
    g_map = zeros(Int32, L, L)
    
    i = 0
        
    Is = [2*rand(1:L), 2*rand(1:L)]
    Ms = Is
    dist_M_to_I = [0,0]
    Z_count = 0
    had_worm = 0
    in_Z_sector = true
    #these conditions set parameters such that the first thing done is to start a worm
    
    while i <= nv
        
        q_c = rand()
        
        if in_Z_sector
            #hey, you're in the Z-sector
            #choose an I=M
            #if q_c < kappa, you then become in the G sector
            #this is the start of a worm
            if q_c < kappa
                #create_worm
                Is = [2*rand(1:L), 2*rand(1:L)]
                Ms = Is
                in_Z_sector = false
                had_worm = 0
            end
            
            #since you were just in the Z sector, with closed worms
            #then measure the winding number
            winding = 0.5*((dist_M_to_I[1]/L)^2 + (dist_M_to_I[2]/L)^2)
            accum_winding += winding
            accum_winding = accum_winding/length(accum_w_list)
            #add the accumulated winding to a running average list
            push!(accum_w_list, accum_winding) 
            
        else
            #hey, you actually were not in the Z sector
            #but rather in the G sector - you need to move the worm
            #move a worm
            Ms, dir_link, all_map = move_worm(Ms, all_map, ratio)
            dist_M_to_I += dir_link
            if dir_link != [0,0]
                gs_map = ones(Int32, 2) + mod.(dist_M_to_I, L)
                g_map[CartesianIndex(gs_map[1], gs_map[2])] += 1
                had_worm = 1
            end 
            
            #did you end up on I=M?
            #if so, ask whether you'd like to return to the Z sector
            #this is also wher you count the number of times you returned to the Z sector after creating a worm of a finite length
            
            if Is == Ms
                if q_c < 1/kappa
                    #annihilate_worm
                    in_Z_sector = true
                    Z_count += 1*(had_worm) #do not count Z configs if you did not update the worm
                end
            end
        end
        
        #counter
        i += 1
        
    end
        
    return Z_count, accum_w_list, g_map
end



sweep_v3 (generic function with 1 method)

In [26]:
####
#mc routine
####
T = 0.1
β = 1/T
ratio = 1/(2*β)
@printf "Temperature %.2f" T
@printf "\n"

all_map = zeros(Int32,2*L, 2*L) 

#first, thermalize
sweep_v3((10^4)*L^2, ratio, all_map, 1.0)
Z_count, accum_w_list, g_map_vals = sweep_v3((10^5)*L^2, ratio, all_map, 1.0)

w_length = length(accum_w_list)
#w_t = T*mean(accum_w_list[Int(w_length/3):end])
final_w = accum_w_list[floor(Int, 2*w_length/3):end]*T
w_t = final_w[end]
w_t_std = (maximum(final_w) - minimum(final_w))/2
@printf "Stiffness %.3f ± %.3f" w_t w_t_std
@printf "\n"
energy = -(g_map_vals[1,2] + g_map_vals[2,1] + g_map_vals[1,L] + g_map_vals[L,1])/(2*Z_count)
@printf "Energy %.2f" energy


Temperature 0.10
Stiffness 1.247 ± 0.017
Energy -3.90